# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 27 2023 11:48AM,258227,10,9212681,"StellaLife, Inc.",Released
1,Feb 27 2023 11:37AM,258219,10,0086433677,ISDIN Corporation,Released
2,Feb 27 2023 11:37AM,258219,10,0086433683,ISDIN Corporation,Released
3,Feb 27 2023 11:37AM,258219,10,0086433669,ISDIN Corporation,Released
4,Feb 27 2023 11:37AM,258219,10,0086433670,ISDIN Corporation,Released
5,Feb 27 2023 11:37AM,258219,10,0086433672,ISDIN Corporation,Released
6,Feb 27 2023 11:37AM,258219,10,0086433674,ISDIN Corporation,Released
7,Feb 27 2023 11:37AM,258219,10,0086433673,ISDIN Corporation,Released
8,Feb 27 2023 11:37AM,258219,10,0086433678,ISDIN Corporation,Released
9,Feb 27 2023 11:37AM,258219,10,0086433657,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,258219,Released,61
39,258220,Released,5
40,258221,Released,25
41,258226,Executing,1
42,258227,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258219,NaN,61.0
258220,NaN,5.0
258221,NaN,25.0
258226,1.0,NaN
258227,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258149,0.0,18.0
258153,1.0,30.0
258162,0.0,58.0
258166,17.0,1.0
258169,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258149,0,18
258153,1,30
258162,0,58
258166,17,1
258169,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258149,0,18
1,258153,1,30
2,258162,0,58
3,258166,17,1
4,258169,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258149,,18
1,258153,1,30
2,258162,,58
3,258166,17,1
4,258169,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc."
1,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation
62,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC"
87,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.
92,Feb 27 2023 11:12AM,258218,10,"Citieffe, Inc."
93,Feb 27 2023 11:08AM,258217,10,ISDIN Corporation
101,Feb 27 2023 11:04AM,258216,10,ISDIN Corporation
106,Feb 27 2023 10:54AM,258214,10,Bio-PRF
130,Feb 27 2023 10:51AM,258215,10,Methapharm-G
131,Feb 27 2023 10:48AM,258213,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc.",,1
1,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation,,61
2,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC",,25
3,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.,,5
4,Feb 27 2023 11:12AM,258218,10,"Citieffe, Inc.",1,
5,Feb 27 2023 11:08AM,258217,10,ISDIN Corporation,,8
6,Feb 27 2023 11:04AM,258216,10,ISDIN Corporation,,5
7,Feb 27 2023 10:54AM,258214,10,Bio-PRF,3,21
8,Feb 27 2023 10:51AM,258215,10,Methapharm-G,,1
9,Feb 27 2023 10:48AM,258213,10,ISDIN Corporation,,23


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc.",1,
1,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation,61,
2,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC",25,
3,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.,5,
4,Feb 27 2023 11:12AM,258218,10,"Citieffe, Inc.",,1
5,Feb 27 2023 11:08AM,258217,10,ISDIN Corporation,8,
6,Feb 27 2023 11:04AM,258216,10,ISDIN Corporation,5,
7,Feb 27 2023 10:54AM,258214,10,Bio-PRF,21,3
8,Feb 27 2023 10:51AM,258215,10,Methapharm-G,1,
9,Feb 27 2023 10:48AM,258213,10,ISDIN Corporation,23,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc.",1,
1,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation,61,
2,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC",25,
3,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.,5,
4,Feb 27 2023 11:12AM,258218,10,"Citieffe, Inc.",,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc.",1.0,NaN
1,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation,61.0,NaN
2,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC",25.0,NaN
3,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.,5.0,NaN
4,Feb 27 2023 11:12AM,258218,10,"Citieffe, Inc.",NaN,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 27 2023 11:48AM,258227,10,"StellaLife, Inc.",1.0,0.0
1,Feb 27 2023 11:37AM,258219,10,ISDIN Corporation,61.0,0.0
2,Feb 27 2023 11:26AM,258221,19,"Innogenix, LLC",25.0,0.0
3,Feb 27 2023 11:23AM,258220,10,Eywa Pharma Inc.,5.0,0.0
4,Feb 27 2023 11:12AM,258218,10,"Citieffe, Inc.",0.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,4905796,357.0,8.0
15,774611,83.0,61.0
16,516399,2.0,3.0
18,1032691,4.0,0.0
19,1549221,30.0,17.0
20,258196,0.0,1.0
22,258226,0.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,4905796,357.0,8.0
1,15,774611,83.0,61.0
2,16,516399,2.0,3.0
3,18,1032691,4.0,0.0
4,19,1549221,30.0,17.0
5,20,258196,0.0,1.0
6,22,258226,0.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,357.0,8.0
1,15,83.0,61.0
2,16,2.0,3.0
3,18,4.0,0.0
4,19,30.0,17.0
5,20,0.0,1.0
6,22,0.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,357.0
1,15,Released,83.0
2,16,Released,2.0
3,18,Released,4.0
4,19,Released,30.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,22
Status,,,,,,,
Executing,8.0,61.0,3.0,0.0,17.0,1.0,1.0
Released,357.0,83.0,2.0,4.0,30.0,0.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,22
0,Executing,8.0,61.0,3.0,0.0,17.0,1.0,1.0
1,Released,357.0,83.0,2.0,4.0,30.0,0.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,22
0,Executing,8.0,61.0,3.0,0.0,17.0,1.0,1.0
1,Released,357.0,83.0,2.0,4.0,30.0,0.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()